In [1]:
import tensorflow as tf
import numpy as np
import seaborn as sns
import os 
import cv2 # For reading the image
import skimage.io 
import matplotlib.pylab    as plt
plt.rcParams['image.cmap'] = 'gray'
%matplotlib inline  
# This will display the image inside the notebook

In [3]:
from natsort import natsorted
from sklearn import preprocessing # For whitening
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [5]:
# Size of the images
rows = 512
cols = 512

In [7]:
# Read the images in Std folder and make npy data
# Read the image directory, construct a tensor and save to npy array
imageDir = './Std/'
images = []
for dirname, subdirList, fileList in os.walk(imageDir):
    # Sort the jpg file numerically
    fileList = natsorted(fileList)
    for f in fileList:
        if f.endswith('png'):
            filename = os.path.join(dirname, f)
            # Print the image file name and read by using opencv
            #print f
            print filename
            image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
                
            # Append to the images list
            images.append(image)
# Conver image no numpy array
images = np.array(images)
print images.shape
# Reshape the image to 
images = np.reshape(images, (-1, rows, cols))

# Save the images to numpy array
np.save('std.npy', images)

# Save the image to tif file
skimage.io.imsave('std.tif', images)

./Std/barbara.grey.png
./Std/bridge.grey.png
./Std/kiel.grey.png
./Std/lena.grey.png
./Std/mandrill.png
./Std/peppers.png
(6, 512, 512)


In [ ]:
# Implementation of Convolutional Sparse Coding Layer
class ConvSparseCodingLayer:
    
    def __init__(self):
        # init method to make the graph
        pass
    def cfft(src_ph):
        # Perform custom forward Fourier transform
        pass
    def cift(src_ph):
        # Perform custom inverse Fourier transform
        pass
    def testFourierTransform():
        pass
    def testBoundaryCrop():
        pass
    def testZeroPadding():
        pass